In [3]:
import json
import os

import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import InterpolationMode
from tqdm import tqdm


DEFAULT_CONFIG = {
    # Supports: "path/img.jpg Label" OR "path/img.jpg\tLabel"
    "data_file": "/home/metythorn/konai/services/ocr-service/data/processed/text-recognition/ocr_dataset_v2.txt",
    "img_height": 128,
    "patch_size": 16,
    "dim_feedforward": 2048,
    "dropout": 0.1,
    "augment": True,
    "max_decode_len": 150,
    "quiet": False,
    "device": "cuda:0" if torch.cuda.is_available() else "cpu",
}

# Adjust epoch base on ur GPU VRAM (I'm using RTX 5090 32GB)
MODEL_VARIANTS = {
    "small": {
        "img_width": 320,
        "batch_size": 128,
        "lr": 1e-4,
        "epochs": 10,
        "d_model": 384,
        "nhead": 6,
        "num_layers": 6,
        "patch_size": 16,
        "dim_feedforward": 1536,
        "checkpoint_dir": "checkpoints_small",
    },
    "base": {
        "img_width": 384,
        "batch_size": 90,
        "lr": 8e-5,
        "epochs": 8,
        "d_model": 768,
        "nhead": 12,
        "num_layers": 10,
        "patch_size": 16,
        "dim_feedforward": 3072,
        "checkpoint_dir": "checkpoints_base",
    },
    "large": {
        "img_width": 384,
        "batch_size": 64,
        "lr": 6e-5,
        "epochs": 8,
        "d_model": 1024,
        "nhead": 16,
        "num_layers": 12,
        "patch_size": 16,
        "dim_feedforward": 4096,
        "checkpoint_dir": "checkpoints_large",
    },
}

CONFIG = {}

In [4]:
def export_config(vocab, path=None):
    return None